In [147]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pandas as pd
from scipy import interp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
#from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn import cross_validate, metrics
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
from sklearn import metrics
import sklearn.tree as tree
datatotal=pd.read_csv("Perfect.csv")
X=datatotal.drop(['Y'],axis=1)
#datatotal=datatotal.drop(['Orthostatis'],axis=1)
#X["X26"]=X["X26"]-1
#X["X27"]=X["X27"]-1
#X["X28"]=X["X28"]-1
#X=preprocessing.scale(X)
n=X.shape[0]
Y=datatotal["Y"]


In [148]:
X.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20
0,1,42,22.06,76,4.72,494,4.17,1.09,1.6,1.94,157.23,29.65,0.23,64,57,0,0,0,0,0
1,1,39,23.91,78,5.13,382,4.49,2.17,1.1,2.32,92.39,15.59,1.33,107,71,0,0,0,0,0
2,0,56,23.26,72,5.39,381,5.57,1.67,1.5,3.09,87.26,20.89,26.78,131,79,0,0,0,0,0
3,0,53,23.27,73,6.10,412,6.37,2.34,1.3,3.70,14.60,12.85,0.20,131,78,0,0,0,0,0
4,0,48,18.75,63,5.38,393,3.86,0.88,1.7,1.73,45.12,5.54,14.97,94,64,0,0,0,0,0


In [149]:
X['X2']=preprocessing.scale(X['X2'])
X['X3']=preprocessing.scale(X['X3'])
X['X4']=preprocessing.scale(X['X4'])
X['X5']=preprocessing.scale(X['X5'])
X['X6']=preprocessing.scale(X['X6'])
X['X7']=preprocessing.scale(X['X7'])
X['X8']=preprocessing.scale(X['X8'])
X['X9']=preprocessing.scale(X['X9'])
X['X10']=preprocessing.scale(X['X10'])
X['X11']=preprocessing.scale(X['X11'])
X['X12']=preprocessing.scale(X['X12'])
X['X13']=preprocessing.scale(X['X13'])
X['X14']=preprocessing.scale(X['X14'])
X['X15']=preprocessing.scale(X['X15'])

In [150]:
X.shape

(150, 20)

In [151]:
X=X.dropna(axis = 0)
Y=Y.dropna(axis=0)

In [152]:
X=X.iloc[:,[0,2,5,9,11,12,13,14]]

In [153]:
Y.shape

(150,)

In [154]:
X.shape

(150, 8)

In [155]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.datasets import fetch_openml
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [83]:
#X, Y = SMOTE(k_neighbors=20,sampling_strategy=0.9).fit_resample(X, Y)

In [142]:
dtc = tree.DecisionTreeClassifier() #Setup 10-fold CV
seed=888
kf = KFold(n_splits=10, shuffle=True,random_state=seed)
matrix = np.matrix('0 0; 0 0') #Generating confusion matrix
TN=0
FP=0
FN=0
TP=0
AUC=0
for train_index, test_index in kf.split(X): #Iterating through each fold of CV
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    X_train, Y_train = SMOTE(k_neighbors=20,sampling_strategy=0.9).fit_resample(X_train, Y_train)
    dtc_cv = DecisionTreeClassifier(max_depth=3,random_state=0)
    #ros = RandomOverSampler(random_state=0,sampling_strategy=1.5)
   # X_train, Y_train = SMOTE(k_neighbors=20).fit_resample(X_train, Y_train)
    #X_train, Y_train = ros.fit_resample(X_train, Y_train)
    dtc_cv.fit(X_train, Y_train)
    Y_pred = dtc_cv.predict(X_test)
    matrix = np.add(matrix, confusion_matrix(Y_test, Y_pred)) #Build model and evaluate with testing data
    tn, fp, fn, tp=confusion_matrix(np.squeeze(Y_test), np.squeeze(Y_pred)).ravel()
    auc=roc_auc_score(Y_test, dtc_cv.predict_proba(X_test)[:, 1])
    TN+=tn
    FP+=fp
    FN+=fn
    TP+=tp
    AUC+=auc
acc=(TN+TP)/(TN+TP+FP+FN)  
precision=TP/(TP+FP)
recall=TP/(TP+FN)
f1=2/(1/precision+1/recall)
print("ACC",acc,"Precision",precision,"Recall",recall,"F1",f1,"AUC",AUC/10)

ACC 0.64 Precision 0.33783783783783783 Recall 0.8333333333333334 F1 0.4807692307692307 AUC 0.7287115662115662


In [85]:
from sklearn import svm
seed=666
kf = KFold(n_splits=10, shuffle=True,random_state=seed)
matrix = np.matrix('0 0; 0 0') #Generating confusion matrix
TN=0
FP=0
FN=0
TP=0
AUC=0
for train_index, test_index in kf.split(X): #Iterating through each fold of CV
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    X_train, Y_train = SMOTE(k_neighbors=20,sampling_strategy=0.9).fit_resample(X_train, Y_train)
    dtc_cv = svm.SVC(probability=True)
    #ros = RandomOverSampler(random_state=0,sampling_strategy=1.5)
   # X_train, Y_train = SMOTE(k_neighbors=20).fit_resample(X_train, Y_train)
    #X_train, Y_train = ros.fit_resample(X_train, Y_train)
    dtc_cv.fit(X_train, Y_train)
    Y_pred = dtc_cv.predict(X_test)
    matrix = np.add(matrix, confusion_matrix(Y_test, Y_pred)) #Build model and evaluate with testing data
    tn, fp, fn, tp=confusion_matrix(np.squeeze(Y_test), np.squeeze(Y_pred)).ravel()
    auc=roc_auc_score(Y_test, dtc_cv.predict_proba(X_test)[:, 1])
    TN+=tn
    FP+=fp
    FN+=fn
    TP+=tp
    AUC+=auc
acc=(TN+TP)/(TN+TP+FP+FN)  
precision=TP/(TP+FP)
recall=TP/(TP+FN)
f1=2/(1/precision+1/recall)
print("ACC",acc,"Precision",precision,"Recall",recall,"F1",f1,"AUC",AUC/10)

ACC 0.6933333333333334 Precision 0.23333333333333334 Recall 0.23333333333333334 F1 0.23333333333333334 AUC 0.6475177045177046


In [173]:
seed=666
kf = KFold(n_splits=10, shuffle=True,random_state=seed)
matrix = np.matrix('0 0; 0 0') #Generating confusion matrix
TN=0
FP=0
FN=0
TP=0
AUC=0
for train_index, test_index in kf.split(X): #Iterating through each fold of CV
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    X_train, Y_train = SMOTE(k_neighbors=20,sampling_strategy=0.9).fit_resample(X_train, Y_train)
    dtc_cv = GradientBoostingClassifier(n_estimators=500, max_depth=2, max_features='sqrt',learning_rate=0.001)
    #ros = RandomOverSampler(random_state=0,sampling_strategy=1.5)
   # X_train, Y_train = SMOTE(k_neighbors=20).fit_resample(X_train, Y_train)
    #X_train, Y_train = ros.fit_resample(X_train, Y_train)
    dtc_cv.fit(X_train, Y_train)
    Y_pred = dtc_cv.predict(X_test)
    matrix = np.add(matrix, confusion_matrix(Y_test, Y_pred)) #Build model and evaluate with testing data
    tn, fp, fn, tp=confusion_matrix(np.squeeze(Y_test), np.squeeze(Y_pred)).ravel()
    auc=roc_auc_score(Y_test, dtc_cv.predict_proba(X_test)[:, 1])
    TN+=tn
    FP+=fp
    FN+=fn
    TP+=tp
    AUC+=auc
acc=(TN+TP)/(TN+TP+FP+FN)  
precision=TP/(TP+FP)
recall=TP/(TP+FN)
f1=2/(1/precision+1/recall)
print("ACC",acc,"Precision",precision,"Recall",recall,"F1",f1,"AUC",AUC/10)

ACC 0.68 Precision 0.2857142857142857 Recall 0.4 F1 0.3333333333333333 AUC 0.7070531690531692


In [176]:
seed=666
kf = KFold(n_splits=10, shuffle=True,random_state=seed)
matrix = np.matrix('0 0; 0 0') #Generating confusion matrix
TN=0
FP=0
FN=0
TP=0
AUC=0
for train_index, test_index in kf.split(X): #Iterating through each fold of CV
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    X_train, Y_train = SMOTE(k_neighbors=20,sampling_strategy=0.8).fit_resample(X_train, Y_train)
    dtc_cv = RandomForestClassifier(n_estimators=500, max_depth=2,max_features='sqrt',random_state=0)
    #ros = RandomOverSampler(random_state=0,sampling_strategy=1.5)
   # X_train, Y_train = SMOTE(k_neighbors=20).fit_resample(X_train, Y_train)
    #X_train, Y_train = ros.fit_resample(X_train, Y_train)
    dtc_cv.fit(X_train, Y_train)
    Y_pred = dtc_cv.predict(X_test)
    matrix = np.add(matrix, confusion_matrix(Y_test, Y_pred)) #Build model and evaluate with testing data
    tn, fp, fn, tp=confusion_matrix(np.squeeze(Y_test), np.squeeze(Y_pred)).ravel()
    auc=roc_auc_score(Y_test, dtc_cv.predict_proba(X_test)[:, 1])
    TN+=tn
    FP+=fp
    FN+=fn
    TP+=tp
    AUC+=auc
acc=(TN+TP)/(TN+TP+FP+FN)  
precision=TP/(TP+FP)
recall=TP/(TP+FN)
f1=2/(1/precision+1/recall)
print("ACC",acc,"Precision",precision,"Recall",recall,"F1",f1,"AUC",AUC/10)

ACC 0.68 Precision 0.30434782608695654 Recall 0.4666666666666667 F1 0.3684210526315789 AUC 0.7480538350538352


In [171]:
seed=666
kf = KFold(n_splits=10, shuffle=True,random_state=seed)
matrix = np.matrix('0 0; 0 0') #Generating confusion matrix
TN=0
FP=0
FN=0
TP=0
AUC=0
for train_index, test_index in kf.split(X): #Iterating through each fold of CV
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    X_train, Y_train = SMOTE(k_neighbors=10,sampling_strategy=0.9).fit_resample(X_train, Y_train)
    base_estimator=DecisionTreeClassifier(max_depth=5,random_state=0)
    dtc_cv = AdaBoostClassifier(base_estimator,learning_rate=0.0001,n_estimators=500,random_state=0)
    #ros = RandomOverSampler(random_state=0,sampling_strategy=1.5)
   # X_train, Y_train = SMOTE(k_neighbors=20).fit_resample(X_train, Y_train)
    #X_train, Y_train = ros.fit_resample(X_train, Y_train)
    dtc_cv.fit(X_train, Y_train)
    Y_pred = dtc_cv.predict(X_test)
    matrix = np.add(matrix, confusion_matrix(Y_test, Y_pred)) #Build model and evaluate with testing data
    tn, fp, fn, tp=confusion_matrix(np.squeeze(Y_test), np.squeeze(Y_pred)).ravel()
    auc=roc_auc_score(Y_test, dtc_cv.predict_proba(X_test)[:, 1])
    TN+=tn
    FP+=fp
    FN+=fn
    TP+=tp
    AUC+=auc
acc=(TN+TP)/(TN+TP+FP+FN)  
precision=TP/(TP+FP)
recall=TP/(TP+FN)
f1=2/(1/precision+1/recall)
print("ACC",acc,"Precision",precision,"Recall",recall,"F1",f1,"AUC",AUC/10)

ACC 0.68 Precision 0.30434782608695654 Recall 0.4666666666666667 F1 0.3684210526315789 AUC 0.7377295482295483


In [68]:
import xgboost as xgb
seed=666
kf = KFold(n_splits=10, shuffle=True,random_state=seed)
matrix = np.matrix('0 0; 0 0') #Generating confusion matrix
TN=0
FP=0
FN=0
TP=0
AUC=0
for train_index, test_index in kf.split(X): #Iterating through each fold of CV
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    X_train, Y_train = SMOTE(k_neighbors=20,sampling_strategy=0.9).fit_resample(X_train, Y_train)
    #base_estimator=DecisionTreeClassifier(max_depth=5,random_state=0)
    dtc_cv = xgb.XGBClassifier(max_depth=3, n_estimators=600, learning_rate=0.01 ,min_child_weight=6,subsample=0.737,reg_alpha=1e-2)
    #ros = RandomOverSampler(random_state=0,sampling_strategy=1.5)
   # X_train, Y_train = SMOTE(k_neighbors=20).fit_resample(X_train, Y_train)
    #X_train, Y_train = ros.fit_resample(X_train, Y_train)
    dtc_cv.fit(X_train, Y_train)
    Y_pred = dtc_cv.predict(X_test)
    matrix = np.add(matrix, confusion_matrix(Y_test, Y_pred)) #Build model and evaluate with testing data
    tn, fp, fn, tp=confusion_matrix(np.squeeze(Y_test), np.squeeze(Y_pred)).ravel()
    auc=roc_auc_score(Y_test, dtc_cv.predict_proba(X_test)[:, 1])
    TN+=tn
    FP+=fp
    FN+=fn
    TP+=tp
    AUC+=auc
acc=(TN+TP)/(TN+TP+FP+FN)  
precision=TP/(TP+FP)
recall=TP/(TP+FN)
f1=2/(1/precision+1/recall)
print("ACC",acc,"Precision",precision,"Recall",recall,"F1",f1,"AUC",AUC/10)

/root/miniconda3/envs/py36/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:33:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/root/miniconda3/envs/py36/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:33:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/root/miniconda3/envs/py36/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[00:33:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/root/miniconda3/envs/py36/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:33:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/root/miniconda3/envs/py36/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:33:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ACC 0.7133333333333334 Precision 0.35555555555555557 Recall 0.5333333333333333 F1 0.4266666666666667 AUC 0.7447064047064048


In [69]:
f1=2/(1/precision+1/0.56)

In [70]:
f1

0.4349514563106796